In [ ]:
# Plotting all the cl_pt spectra with error bars
import os
# ─── environment setup ─────────────────────────────────────────────
import env_config
# ────────────────────────────────────────────────────────────────────

import numpy as np
from binner_sims import ffp10_binner_phiT as binner_sims
import thesis_plot_style as tps

# your parameter files
import parfiles.noNoise.parfile            as par_nn
import parfiles.noNoise.parfile_delensed   as par_nn_del
import parfiles.noNoise.parfile_delensed_qest    as par_nn_qest
import parfiles.noNoise.parfile_delensed_PolQEST as par_nn_polq

import parfiles.Noise.parfile            as par_n
import parfiles.Noise.parfile_delensed   as par_n_del
import parfiles.Noise.parfile_delensed_qest    as par_n_qest
import parfiles.Noise.parfile_delensed_PolQEST as par_n_polq

# common settings
bin_type = "fullA_10"

# define scenarios as (parfile, label, kPhi)
scenarios = [
    (par_nn,     "MV: lensed",                    'p'),
    (par_nn_del, r"MV: delensed with input $\kappa_{LM}$",   'p'),
    (par_nn_qest,"MV: int. delensed with MV-QEST",'p'),
    (par_nn_polq,"TT: int. delensed with Pol-QEST",'ptt'),
]
noisy_scenarios = [
    (par_n,      "MV: lensed",                    'p'),
    (par_n_del,  r"MV: delensed with input $\kappa_{LM}$",   'p'),
    (par_n_qest,"MV: int. delensed with MV-QEST",'p'),
    (par_n_polq,"TT: int. delensed with Pol-QEST",'ptt'),
]

tps.apply_style("single")

# ─── Noiseless figure ────────────────────────────────────────
with tps.context_figure(filename="fig_clpt_noiseless.png") as (fig, ax):
    # 1) plot fiducial theory
    b0       = binner_sims('p', par_nn, bin_type)
    cl_fid   = b0.fiducial_cl_scaled(False)
    ell_full = np.arange(len(cl_fid))
    ax.plot(ell_full, cl_fid,
            'k--', lw=1.6, label="fiducial")

    # 2) binned MC curves + error bars
    ell = b0.bin_lavs
    for ci, (pf, label, kPhi) in enumerate(scenarios):
        b = binner_sims(kPhi, pf, bin_type)

        # returns binned ⟨Cℓ⟩ and σ(⟨Cℓ⟩)
        cl_mean, cl_err = b.get_cL_PHI_T_with_error(scaled=True)

        color = tps.CB_PALETTE[ci+2]
        ax.errorbar(ell, cl_mean,
                    yerr=cl_err,
                    fmt='o-', lw=1.6, capsize=3,
                    color=color,
                    label=label)

    ax.set(xlabel=r"$\ell$",
           ylabel=r"$10^2\,\ell^3\,C_\ell^{\phi T}\ [\mu{\rm K}]$",
           title="Noiseless $C_\ell^{\phi T}$ Estimates")
    ax.set_xlim([0, 105])
    ax.legend(loc="upper right")

# ─── Noisy figure ────────────────────────────────────────────
with tps.context_figure(filename="fig_clpt_noisy.png") as (fig, ax):
    # 1) plot fiducial theory
    b0       = binner_sims('p', par_n, bin_type)
    cl_fid   = b0.fiducial_cl_scaled(False)
    ell_full = np.arange(len(cl_fid))
    ax.plot(ell_full, cl_fid,
            'k--', lw=1.6, label="fiducial")

    # 2) binned MC curves + error bars (no extra calibration needed here)
    ell = b0.bin_lavs
    for ci, (pf, label, kPhi) in enumerate(noisy_scenarios):
        b = binner_sims(kPhi, pf, bin_type)

        cl_mean, cl_err = b.get_cL_PHI_T_with_error(scaled=True)

        color = tps.CB_PALETTE[ci+2]
        ax.errorbar(ell, cl_mean,
                    yerr=cl_err,
                    fmt='o-', lw=1.6, capsize=3,
                    color=color,
                    label=label)

    ax.set(xlabel=r"$\ell$",
           ylabel=r"$10^2\,\ell^3\,C_\ell^{\phi T}\ [\mu{\rm K}]$",
           title="Noisy $C_\ell^{\phi T}$ Estimates")
    ax.set_xlim([0, 105])
    ax.legend(loc="upper right")


Using lenspyx alm2map


In [ ]:
#!/usr/bin/env python3
import os, numpy as np, matplotlib.pyplot as plt
# ─── environment setup ─────────────────────────────────────────────
import env_config
# ────────────────────────────────────────────────────────────────────

from binner_sims import binner_sims              # PP binner for Wℓ
from binner_sims import ffp10_binner_phiT as PT   # φ–T binner
import thesis_plot_style as tps

# ─── parameter files ─────────────────────────────────────────────────────
import parfiles.noNoise.parfile                       as par_nn
import parfiles.noNoise.parfile_delensed_qest         as par_nn_qest
import parfiles.noNoise.parfile_delensed_PolQEST      as par_nn_polq

# ─── environment + common settings ────────────────────────────────────────
bin_type = "fullA_10"

tps.apply_style("single")
cmap = tps.CB_PALETTE

# ─── Noiseless: Wℓ (Pol-QE & MV-QE) + two φ–T efficiencies ──────────────
with tps.context_figure(filename="fig_wf2_eff_phiT_noiseless.png") as (fig, ax):
    
    ax.axvspan(25, 35, color="gray", alpha=0.2, zorder=0)
    
    # 1) Wiener filters
    bw_pol = binner_sims("p_p", "p_p", par_nn,      bin_type)
    bw_mv  = binner_sims("p",   "p",   par_nn,      bin_type)
    ell_W  = bw_pol.bin_lavs

    ax.plot(np.arange(2049), bw_mv.get_WF()**.5,
            color=cmap[2], lw=1.6,
            label="Wiener $W_\\ell$ (MV-QE)")
    
    ax.plot(np.arange(2049), bw_pol.get_WF()**.5,
            color=cmap[4], lw=1.6,
            label="Wiener $W_\\ell$ (Pol-QE)")

    # 2) φ–T baseline & delensed scenarios
    b_base  = PT("p",   par_nn,     bin_type)
    C0, σ0   = b_base.get_cL_PHI_T_with_error(scaled=True)


    # a) MV-internal delensed
    b_mvint = PT("p",   par_nn_qest,bin_type)
    C1, σ1   = b_mvint.get_cL_PHI_T_with_error(scaled=True)

    # b) Pol-QEST delensed
    b_poltd = PT("ptt", par_nn_polq,bin_type)
    C2, σ2   = b_poltd.get_cL_PHI_T_with_error(scaled=True)

    # efficiency #1: MV-int
    eff1   = 1.0 - C1/C0
    err1   = np.sqrt((σ1/C0)**2 + (C1*σ0/C0**2)**2)

    # efficiency #2: Pol-QEST
    eff2   = 1.0 - C2/C0
    err2   = np.sqrt((σ2/C0)**2 + (C2*σ0/C0**2)**2)

    ell = b_base.bin_lavs
    ax.errorbar(ell, eff1, yerr=err1,
                fmt='o-', capsize=3, lw=1.4,
                color=cmap[3],
                label= r"$\epsilon$" + "$_{\\rm MV\\!\\to\\!MV\\text{-}QEST}$")
    ax.errorbar(ell, eff2, yerr=err2,
                fmt='o-', capsize=3, lw=1.4,
                color=cmap[5],
                label= r"$\epsilon$" + "$_{\\rm TT\\!\\to\\!Pol\\text{-}QEST}$")
    ax.hlines(0, 0,300,'grey', '--', alpha=0.3)
    ax.set(xlabel=r"$\ell$",
           ylabel="Dimensionless",
           title="Wiener Filters & φ–T Efficiencies\n(Noiseless)")
    ax.set_xlim(0,105)
    ax.set_ylim(-0.1,1)
    ax.legend(loc="lower right", frameon=False)


In [ ]:
#!/usr/bin/env python3
import os, numpy as np, matplotlib.pyplot as plt
# ─── environment setup ─────────────────────────────────────────────
import env_config
# ────────────────────────────────────────────────────────────────────

from binner_sims import binner_sims
from binner_sims import ffp10_binner_phiT as PT
import thesis_plot_style as tps

# ─── parameter files ─────────────────────────────────────────────────────
import parfiles.Noise.parfile                       as par_n
import parfiles.Noise.parfile_delensed_qest        as par_n_qest
import parfiles.Noise.parfile_delensed_PolQEST     as par_n_polq

bin_type = "fullA_10"

tps.apply_style("single")

cmap = tps.CB_PALETTE

# ─── Noisy: Wℓ (Pol-QE & MV-QE) + two φ–T efficiencies ─────────────────
with tps.context_figure(filename="fig_wf2_eff_phiT_noisy.png") as (fig, ax):
    
    ax.axvspan(25, 35, color="gray", alpha=0.2, zorder=0)

    # 1) Wiener filters
    bw_pol = binner_sims("p_p","p_p", par_n,      bin_type)
    bw_mv  = binner_sims("p",  "p",   par_n,      bin_type)
    ell_W  = np.arange(4097)


    ax.plot(ell_W, bw_mv.get_WF()**.5,
            color=cmap[2], lw=1.6,
            label="Wiener $W_\\ell$ (MV-QE)")
    ax.plot(ell_W, bw_pol.get_WF()**.5,
            color=cmap[4], lw=1.6,
            label="Wiener $W_\\ell$ (Pol-QE)")

    # 2) φ–T baseline & delensed scenarios
    b_base  = PT("p",   par_n,      bin_type)
    C0, σ0   = b_base.get_cL_PHI_T_with_error(scaled=True)

    b_mvint = PT("p",   par_n_qest, bin_type)
    C1, σ1   = b_mvint.get_cL_PHI_T_with_error(scaled=True)

    b_poltd = PT("ptt", par_n_polq, bin_type)
    C2, σ2   = b_poltd.get_cL_PHI_T_with_error(scaled=True)

    eff1   = 1.0 - C1/C0
    err1   = np.sqrt((σ1/C0)**2 + (C1*σ0/C0**2)**2)

    eff2   = 1.0 - C2/C0
    err2   = np.sqrt((σ2/C0)**2 + (C2*σ0/C0**2)**2)

    ell = b_base.bin_lavs
    ax.errorbar(ell, eff1, yerr=err1,
                fmt='o-', capsize=3, lw=1.4,
                color=cmap[3],
                label= r"$\epsilon$" + "$_{\\rm MV\\!\\to\\!MV\\text{-}QEST}$")
    ax.errorbar(ell, eff2, yerr=err2,
                fmt='o-', capsize=3, lw=1.4,
                color=cmap[5],
                label= r"$\epsilon$" + "$_{\\rm TT\\!\\to\\!Pol\\text{-}QEST}$")

    ax.hlines(0, 0,300,'grey', '--', alpha=0.3)
    ax.set(xlabel=r"$\ell$",
           ylabel="Dimensionless",
           title="Wiener Filters & φ–T Efficiencies\n(Noisy)")
    ax.set_xlim(0,105)
    ax.set_ylim(-0.1,1)
    ax.legend(loc="upper right", frameon=False)


Using lenspyx alm2map
